In [1]:
using LowLevelFEM, LinearAlgebra, SparseArrays
using Tensors
using StaticArrays

In [2]:
gmsh.initialize()
gmsh.open("cube1.geo")

mat = Material("cube", E=2e2)
prob = Problem([mat], type=:VectorField)

Info    : Reading 'cube1.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.000851389s, CPU 0.000852s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 20%] Meshing surface 2 (Transfinite)
Info    : [ 40%] Meshing surface 3 (Transfinite)
Info    : [ 60%] Meshing surface 4 (Transfinite)
Info    : [ 70%] Meshing surface 5 (Transfinite)
Info    : [ 90%] Meshing surface 6 (Transfinite)
Info    : Done meshing 2D (Wall 0.000353656s, CPU 0.000341s)
Info    : Meshing 3D

Problem("cube1", :VectorField, 3, 3, Material[Material("cube", :Hooke, 200.0, 0.3, 115.38461538461537, 76.92307692307692, 166.66666666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 6859, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [3]:
F = TensorField(prob, "cube", [1 0 0; 0 1 0; 0 0 1]);

In [4]:
#Fleft = nodesToElements(elementsToNodes(F), onPhysicalGroup="left")

#Kext = externalTangentFollowerTL(prob; F=Fleft, traction_phName="left", t_spatial=[1, 0, 0])

In [5]:
suppX = BoundaryCondition("left", ux=0, uy=0, uz=0)
suppY = BoundaryCondition("bottom", uy=0)
suppZ = BoundaryCondition("rear", uz=0)


load = BoundaryCondition("right", fy=(x, y, z) -> -(z - 0.5) * 30, fz=(x, y, z) -> (y - 0.5) * 30)

BoundaryCondition("right", nothing, nothing, nothing, nothing, nothing, nothing, nothing, var"#3#4"(), var"#5#6"(), nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing)

In [6]:
μ = mat.μ
λ = mat.λ

115.38461538461537

In [7]:
f_ext0 = loadVector(prob, [load])

nodal VectorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [8]:
showDoFResults(f_ext0, name="f")

1

In [9]:
u = vectorField(prob, "cube", [0, 0, 0])

nodal VectorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [10]:
F = tensorField(prob, "cube", [1 0 0; 0 1 0; 0 0 1])

nodal TensorField
[1.0; 0.0; … ; 0.0; 1.0;;]

In [11]:
I = tensorField(prob, "cube", [1 0 0; 0 1 0; 0 0 1])

nodal TensorField
[1.0; 0.0; … ; 0.0; 1.0;;]

In [12]:
P = tensorField(prob, "cube", [0 0 0; 0 0 0; 0 0 0])
S = tensorField(prob, "cube", [0 0 0; 0 0 0; 0 0 0])

nodal TensorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [13]:
λ0 = λ
μ0 = μ
C1 = [λ0+2μ0 λ0 λ0 0 0 0;
    λ0 λ0+2μ0 λ0 0 0 0;
    λ0 λ0 λ0+2μ0 0 0 0;
    0 0 0 μ0 0 0;
    0 0 0 0 μ0 0;
    0 0 0 0 0 μ0]

6×6 Matrix{Float64}:
 269.231  115.385  115.385   0.0      0.0      0.0
 115.385  269.231  115.385   0.0      0.0      0.0
 115.385  115.385  269.231   0.0      0.0      0.0
   0.0      0.0      0.0    76.9231   0.0      0.0
   0.0      0.0      0.0     0.0     76.9231   0.0
   0.0      0.0      0.0     0.0      0.0     76.9231

In [14]:
function svk_energy2(C, p)
    λ = p.λ
    μ = p.μ

    I = [1.0 0 0; 0 1 0; 0 0 1]

    E = (C - I) / 2

    return (λ * tr(E)^2 + 2μ * dot(E, E)) / 2
end


svk_energy2 (generic function with 1 method)

In [15]:
using StaticArrays

Cst = SMatrix{3,3}([1.0 0 0; 0 1 0; 0 0 1])
LowLevelFEM._tangent_from_energy(svk_energy2, Cst, (λ=λ, μ=μ))

6×6 SMatrix{6, 6, Float64, 36} with indices SOneTo(6)×SOneTo(6):
 269.231  115.385  115.385   0.0      0.0      0.0
 115.385  269.231  115.385   0.0      0.0      0.0
 115.385  115.385  269.231   0.0      0.0      0.0
   0.0      0.0      0.0    76.9231   0.0      0.0
   0.0      0.0      0.0     0.0     76.9231   0.0
   0.0      0.0      0.0     0.0      0.0     76.9231

In [16]:
C1

6×6 Matrix{Float64}:
 269.231  115.385  115.385   0.0      0.0      0.0
 115.385  269.231  115.385   0.0      0.0      0.0
 115.385  115.385  269.231   0.0      0.0      0.0
   0.0      0.0      0.0    76.9231   0.0      0.0
   0.0      0.0      0.0     0.0     76.9231   0.0
   0.0      0.0      0.0     0.0      0.0     76.9231

In [17]:
using StaticArrays

m = [1.0 0 1; 0 1 0; 0 0 1]
c = m' * m
Cst = SMatrix{3,3}(c)
LowLevelFEM._stress_from_energy(svk_energy2, Cst, (λ=λ, μ=μ))

3×3 SMatrix{3, 3, Float64, 9} with indices SOneTo(3)×SOneTo(3):
 57.6923   0.0      76.9231
  0.0     57.6923    0.0
 76.9231   0.0     134.615

In [18]:
F0 = m
I0 = [1.0 0 0; 0 1 0; 0 0 1]
E0 = (F0' * F0 - I0) / 2
S0 = λ * tr(E0) * I0 + 2μ * E0
#P = F * S

3×3 Matrix{Float64}:
 57.6923   0.0      76.9231
  0.0     57.6923    0.0
 76.9231   0.0     134.615

In [19]:
Kmat = materialTangentMatrix(prob, F=F, energy=svk_energy2, params=(λ=λ, μ=μ))

sparse([1, 2, 3, 46, 47, 48, 73, 74, 75, 76  …  20511, 20521, 20522, 20523, 20569, 20570, 20571, 20575, 20576, 20577], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577], [7.214941437163649, 0.7122507122507099, -0.7122507122507103, -0.4108895220006326, -0.17806267806267748, -0.04748338081671256, -2.2171573282684283, 0.35612535612535806, 0.18993352326685378, -0.41088952200063583  …  -1.0940170940171545, 1.4155343563970746e-15, -1.484923295436147e-14, -7.941753719531267, -1.559863349598345e-14, 1.3822276656583199e-14, 72.28616650838896, 1.9498291869979312e-14, 5.773159728050814e-15, 584.7723963279514], 20577, 20577)

In [20]:
C = F' * F
Kgeo = initialStressMatrix(prob, energy=svk_energy2, params=(λ=λ, μ=μ), C=C)

sparse([1, 46, 73, 76, 100, 484, 508, 805, 1375, 1453  …  5832, 5838, 7374, 19599, 19695, 19707, 20511, 20523, 20571, 20577], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577], [-6.0577639582181085e-15, -3.2264621685491715e-16, 2.1425221077808203e-15, -4.943907760358246e-16, 2.8019819880610902e-15, 1.0119077500936618e-15, -3.300067713983044e-15, 5.263760606888135e-16, -1.1791228153415202e-15, -1.6805973146625663e-15  …  -1.0092270983503297e-15, -7.984848858671261e-16, 8.291595734045311e-16, 1.0257400562934199e-15, -2.9370914099432165e-16, -3.7275725515487496e-16, -2.9370914099432993e-16, -3.727572551548857e-16, -5.284330225342806e-15, -4.571255734862617e-15], 20577, 20577)

In [21]:
f_int = internalForceVector(prob, energy=svk_energy2, params=(λ=λ, μ=μ), F=F)

nodal VectorField
[4.202952757135736e-17; 3.929882386451069e-16; … ; 1.4563753103182749e-16; 1.4563753103182556e-16;;]

In [22]:
Kmat0 = materialTangentMatrix(prob, F=F, C=C1)

sparse([1, 2, 3, 46, 47, 48, 73, 74, 75, 76  …  20511, 20521, 20522, 20523, 20569, 20570, 20571, 20575, 20576, 20577], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577], [7.214941437163649, 0.7122507122507099, -0.7122507122507103, -0.4108895220006326, -0.17806267806267748, -0.04748338081671256, -2.2171573282684283, 0.35612535612535806, 0.18993352326685378, -0.41088952200063583  …  -1.0940170940171545, 1.4155343563970746e-15, -1.484923295436147e-14, -7.941753719531267, -1.559863349598345e-14, 1.3822276656583199e-14, 72.28616650838896, 1.9498291869979312e-14, 5.773159728050814e-15, 584.7723963279514], 20577, 20577)

In [23]:
Kgeo0 = initialStressMatrix(prob, S=S)

sparse(Int64[], Int64[], Float64[], 20577, 20577)

In [24]:
f_int0 = internalForceVector(prob, P=P)

nodal VectorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [25]:
norm(Kmat0.A - Kmat.A)

0.0

In [26]:
norm(Kgeo0.A - Kgeo.A)

3.0766424094077427e-12

In [27]:
norm(f_int0.a - f_int.a)

3.230954501580737e-14

In [28]:
#showDoFResults(f_int0, name="f_int_P")
#showDoFResults(f_int, name="f_int_ψ")
#openPostProcessor()
K = nothing
f_int = nothing
f_ext = nothing
free = nothing

## Innen indul az iteráció

In [29]:
n = 10
uu = Vector{VectorField}(undef, n)
for i in 1:10

    Kmat = materialTangentMatrix(prob, F=F, energy=svk_energy2, params=(λ=λ, μ=μ))

    C = F' * F
    Kgeo = initialStressMatrix(prob, energy=svk_energy2, params=(λ=λ, μ=μ), C=C)

    Kint = Kmat + Kgeo
    Fright = nodesToElements(elementsToNodes(F), onPhysicalGroup="right")

    Kext = externalTangentFollower(prob, [load], F=Fright)
    f_int = internalForceVector(prob, energy=svk_energy2, params=(λ=λ, μ=μ), F=F)
    K = Kint - Kext
    FF = nodesToElements(elementsToNodes(F), onPhysicalGroup="right")
    f_ext = loadVector(prob, [load], F=FF)
    Δu = solveField(
        K,
        f_ext - f_int;
        support=[suppX]
    )

    fixed = constrainedDoFs(prob, [suppX])
    free = freeDoFs(prob, [suppX])
    #norm(Δu.a[:, 1])
    r = f_ext - f_int
    println("‖r_free‖  = ", norm(r.a[free, 1]))
    println("‖r_fixed‖ = ", norm(r.a[fixed, 1]))
    println("‖r‖ = ", norm(r.a[:, 1]))
    println("‖Δu‖ = ", norm(Δu.a[:, 1]))

    u += Δu
    uu[i] = copy(u)
    #showDoFResults(u)
    F = I + u ∘ ∇
    E = 0.5 * (F' * F - I)
    S = λ * trace(E) * I + 2μ * E
    P = F * S
    #P = λ * trace(F - I) * I + 2μ * (F - I)

    if norm(Δu.a[:, 1]) < 0.01
        break
    end

end

‖r_free‖  = 0.7372768252552513
‖r_fixed‖ = 9.20414306661775e-15
‖r‖ = 0.7372768252552513
‖Δu‖ = 54.215657539328745
‖r_free‖  = 1697.878718822686
‖r_fixed‖ = 0.590698713409716
‖r‖ = 1697.878821575891
‖Δu‖ = 34.30032456589198
‖r_free‖  = 468.50848494373986
‖r_fixed‖ = 0.8117542546374324
‖r‖ = 468.5091881801769
‖Δu‖ = 10.533554121321696
‖r_free‖  = 117.25625812021656
‖r_fixed‖ = 0.5389526695105925
‖r‖ = 117.25749672551783
‖Δu‖ = 21.403047231673817
‖r_free‖  = 25.400706771429213
‖r_fixed‖ = 0.4450954532262171
‖r‖ = 25.404606166020614
‖Δu‖ = 6.905303491944291
‖r_free‖  = 4.028526232078633
‖r_fixed‖ = 0.38516749040081594
‖r‖ = 4.04689728041216
‖Δu‖ = 2.1350955715924713
‖r_free‖  = 1.3496211809132925
‖r_fixed‖ = 0.3551173319051405
‖r‖ = 1.3955592611527523
‖Δu‖ = 3.2845431518334016
‖r_free‖  = 3.717581115263214
‖r_fixed‖ = 0.34657039676790424
‖r‖ = 3.733700629198536
‖Δu‖ = 1.846102998283744
‖r_free‖  = 0.8547046670316917
‖r_fixed‖ = 0.35476358898043503
‖r‖ = 0.9254065441264364
‖Δu‖ = 4.6356559

In [ ]:
function merge2(uu::Vector{Union{ScalarField,VectorField,TensorField}})
    T = typeof(uu)
    nn = length(uu)
    n = 0
    for i in 1:nn
        n += uu[i].nsteps
    end
    a = zeros(size(uu[1].a, 1), n)
    t = zeros(n)

    a[:, 1:uu[1].nsteps] = uu[1].a
    c = uu[1].nsteps
    t[1:uu[1].nsteps] = c == 1 ? 1 : uu[1].t

    for i in 2:nn
        ns = uu[i].nsteps
        a[:, c+1:c+ns] = uu[i].a
        dt = uu[i-1].nsteps == 1 ? 1 : (uu[i-1].t[end] - uu[i-1].t[end-1])
        t[c+1:c+ns] = uu[i].t .+ dt .+ t[c]
        c += ns
    end
    u = T([], a, t, [], n, uu[1].type, uu[1].model)
    return u
end

merge2 (generic function with 1 method)

In [ ]:
"""
    merge(uu::Vector{<:LowLevelFEM.AbstractField})

Concatenates multiple Fields along a common pseudo-time axis.

Interpretation of `nsteps`:
- If `nsteps == 1`, the Field is treated as an iteration state
  (pseudo-time index: 1, 2, 3, ...).
- If `nsteps > 1`, the Field is treated as a time history, and its
  internal time vector `t` is preserved.

When concatenating multiple time histories, continuity of time is
enforced by shifting each subsequent segment such that it starts
after the previous one. The time increment between segments is
estimated from the last two time points of the previous Field.
If this information is unavailable (`nsteps == 1`), a unit increment
(`Δt = 1`) is assumed.

This function is intended for post-processing and visualization
(e.g. merging iteration histories or successive load/time segments),
not for time-accurate integration.

All input Fields must:
- have the same number of degrees of freedom,
- belong to the same model,
- share the same Field type (ScalarField, VectorField, or TensorField).

Returns a Field of the same concrete type as the first element of `uu`.

# Exaples

u1 = solveDisplacement(problem, load=[load1])   # nsteps = 1
u2 = solveDisplacement(problem, load=[load2])   # nsteps = 1
u3 = solveDisplacement(problem, load=[load3])   # nsteps = 1

u_steps = merge([u1, u2, u3])

# Warning

The time increment between concatenated segments is estimated heuristically.

When a Field has `nsteps == 1`, it is interpreted as an iteration state
and a unit pseudo-time increment (`Δt = 1`) is assumed when concatenating
with subsequent segments.

This heuristic ensures continuity of the time axis but does NOT represent
physical time. The resulting time vector should therefore be used only for
post-processing and visualization, not for time-accurate analysis.
"""
function merge(uu::Vector{<:LowLevelFEM.AbstractField})
    nn = length(uu)
    @assert nn ≥ 1

    # --- összes időlépés
    n = 0
    for u in uu
        n += u.nsteps
    end

    ndofs = size(uu[1].a, 1)
    a = zeros(ndofs, n)
    t = zeros(n)

    # --- első blokk
    ns = uu[1].nsteps
    a[:, 1:ns] .= uu[1].a

    if ns == 1
        t[1] = 1.0                  # iterációs lépés
    else
        t[1:ns] .= uu[1].t          # valódi idő
    end
    c = ns

    # --- további blokkok
    for i in 2:nn
        ui = uu[i]
        ns = ui.nsteps

        a[:, c+1:c+ns] .= ui.a

        # időlépés becslés az előző szegmensből
        if uu[i-1].nsteps > 1
            dt = uu[i-1].t[end] - uu[i-1].t[end-1]
        else
            dt = 1.0
        end

        t[c+1:c+ns] .= ui.t .+ (t[c] + dt)
        c += ns
    end

    # --- visszaépítés: ugyanaz a Field típus, mint az első
    T = typeof(uu[1])
    return T([], a, t, [], n, uu[1].type, uu[1].model)
end


merge (generic function with 3 methods)

In [38]:
u = merge(uu)

nodal VectorField
[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; -1.0833943119584555 -0.8971362613671647 … -0.9955135973038411 -1.0076190199035069; 1.0954859320340784 0.6433035397839105 … -0.12302303139912299 -0.1680397648639144]

In [39]:
showDoFResults(u, name="u_iter")

2

In [40]:
openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0
Fontconfig warning: using without calling FcInit()


In [34]:
Kgeo.A

20577×20577 SparseMatrixCSC{Float64, Int64} with 1167051 stored entries:
⎡⣿⣿⣿⡿⢺⡧⣿⠼⣿⣾⣟⣿⣟⣛⣿⣶⣾⣷⣤⣜⣓⣀⣘⣓⣀⣐⣓⣀⣐⣓⣀⣐⣓⣀⣐⣟⣛⣻⣟⣿⎤
⎢⣿⡿⠻⣦⣼⡇⣿⢠⣤⣴⡶⣯⣽⣷⣯⣭⣭⣭⣭⣭⣭⣭⣭⣽⣿⣿⣿⡿⠿⠿⠷⠶⠶⠶⢶⡶⠶⣶⡶⣦⎥
⎢⠾⡶⠶⠿⢿⣷⡟⠘⣷⢰⡖⣶⣿⣿⣷⠀⠀⣶⠀⠀⣾⠉⠉⣿⠉⠉⣯⠉⠉⡯⠙⠛⡿⠛⠛⡟⠛⠓⡖⠒⎥
⎢⣛⡟⠛⣛⣛⠉⢿⣷⡏⢸⡇⣿⣿⣿⡏⠀⢸⡏⠀⢸⡏⠀⢸⡏⠀⢸⡏⠀⢸⡏⠀⢸⡏⠀⢸⡇⠀⢸⡇⢶⎥
⎢⣻⣿⢀⣿⢙⣛⣋⣉⡿⣯⣁⢻⠉⠉⡛⠿⣭⡅⠀⠀⠁⠀⠀⠁⠀⠈⠁⠀⠈⠁⠀⠈⠁⠀⠈⣁⣀⡈⣁⡈⎥
⎢⣿⣽⡼⣯⢸⣭⣭⣭⣥⣘⠿⣧⡀⠀⣧⣄⣀⣤⣄⣀⣀⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠻⠯⠿⠯⎥
⎢⣿⢹⢷⣿⣿⣿⣿⣿⡇⠀⠀⠈⠻⣦⡀⠉⠉⠉⠉⠛⠛⠛⠛⠿⠿⠿⠶⣶⣶⣶⣦⣤⣤⣤⣄⣀⣀⣀⠀⠀⎥
⎢⢻⣿⡏⣿⠙⠛⠋⠉⣿⡌⠉⢿⡄⠈⠻⣦⡀⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠉⠉⠉⠉⠉⠉⠉⠉⎥
⎢⢾⣿⡇⣿⢠⣤⡶⠶⠇⠿⠀⣼⡇⠀⣤⡈⠻⣦⡈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣀⢿⡇⣿⠀⠀⣀⣀⠀⠀⠀⢹⣧⠀⠈⠻⣦⡈⠻⣦⡈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠙⢸⡇⣿⡞⠛⠋⠉⠁⠀⠀⢸⣿⠀⠀⠀⠈⠻⣦⡈⠻⣦⡈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢶⢸⣇⣿⣧⣤⡶⠶⠄⠀⠀⠈⣿⡄⠀⠀⠀⠀⠈⠻⣦⡈⠻⣦⡈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢀⢸⣿⣿⡇⠀⣀⣀⡀⠀⠀⠀⣿⡇⠀⠀⠀⠀⠀⠀⠈⠻⣦⡈⠻⣦⡈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠙⢸⣿⡿⡏⠛⠋⠉⠁⠀⠀⠀⢸⣧⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡈⠻⣦⡈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢴⢸⣿⡇⡧⡤⡶⠶⠆⠀⠀⠀⢸⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡈⠻⣦⡈⠻⣦⡀⠀⠀⠀⠀⠀⠀⎥
⎢⢀⢸⢹⡇⣷⠀⣀⣀⡀⠀⠀⠀⠈⣿⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡈⠻⣦⡈⠻⣦⡀⠀⠀⠀⠀⎥
⎢⠙⢸⢸⡇⣿⠋⠋⠉⠁⠀⠀⠀⠀⣿⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡈⠻⣦⡈⠻⣦⡀⠀⠀⎥
⎢⣴⢼⢸⡷⣿⠤⠶⠶⠆⢠⣄⠀⠀⢹⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡈⠻⣦⡈⠻⣦⠀⎥
⎢⣿⣸⢸⣧⢿⠀⣀⣀⡀⠸⡿⡆⠀⢸⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡈⠻⣦⡹⣧⎥
⎣⣿⣽⠸⣯⢸⠉⢩⣍⡁⠸⡿⡇⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛⠷⣮⡻⣮⎦

In [35]:
gmsh.finalize()